# Assignment 3 - Named Entity Recognition (NER)

Welcome to the third programming assignment of Course 3. In this assignment, you will learn to build more complicated models with Trax. By completing this assignment, you will be able to: 

- Design the architecture of a neural network, train it, and test it. 
- Process features and represents them
- Understand word padding
- Implement LSTMs
- Test with your own sentence

## Outline
- [Introduction](#0)
- [Part 1:  Exploring the data](#1)
    - [1.1  Importing the Data](#1.1)
    - [1.2  Data generator](#1.2)
		- [Exercise 01](#ex01)
- [Part 2:  Building the model](#2)
	- [Exercise 02](#ex02)
- [Part 3:  Train the Model ](#3)
	- [Exercise 03](#ex03)
- [Part 4:  Compute Accuracy](#4)
	- [Exercise 04](#ex04)
- [Part 5:  Testing with your own sentence](#5)


<a name="0"></a>
# Introduction

We first start by defining named entity recognition (NER). NER is a subtask of information extraction that locates and classifies named entities in a text. The named entities could be organizations, persons, locations, times, etc. 

For example:

<img src = 'ner.png' width="width" height="height" style="width:600px;height:150px;"/>

Is labeled as follows: 

- French: geopolitical entity
- Morocco: geographic entity 
- Christmas: time indicator

Everything else that is labeled with an `O` is not considered to be a named entity. In this assignment, you will train a named entity recognition system that could be trained in a few seconds (on a GPU) and will get around 75% accuracy. Then, you will load in the exact version of your model, which was trained for a longer period of time. You could then evaluate the trained version of your model to get 96% accuracy! Finally, you will be able to test your named entity recognition system with your own sentence.

In [1]:
#!pip -q install trax==1.3.1

import trax 
from trax import layers as tl
import os 
import numpy as np
import pandas as pd


from utils import get_params, get_vocab
import random as rnd

# set random seeds to make this notebook easier to replicate
trax.supervised.trainer_lib.init_random_number_generators(33)

INFO:tensorflow:tokens_length=568 inputs_length=512 targets_length=114 noise_density=0.15 mean_noise_span_length=3.0 


DeviceArray([ 0, 33], dtype=uint32)

In [2]:
import numpy as np
import pandas as pd
import os

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

#from utils import get_params, get_vocab
import random as rnd

<a name="1"></a>
# Part 1:  Exploring the data

We will be using a dataset from Kaggle, which we will preprocess for you. The original data consists of four columns, the sentence number, the word, the part of speech of the word, and the tags.  A few tags you might expect to see are: 

* geo: geographical entity
* org: organization
* per: person 
* gpe: geopolitical entity
* tim: time indicator
* art: artifact
* eve: event
* nat: natural phenomenon
* O: filler word


In [3]:
# display original kaggle data
data = pd.read_csv("ner_dataset.csv", encoding = "ISO-8859-1") 
train_sents = open('data/small/train/sentences.txt', 'r').readline()
train_labels = open('data/small/train/labels.txt', 'r').readline()
print('SENTENCE:', train_sents)
print('SENTENCE LABEL:', train_labels)
print('ORIGINAL DATA:\n', data.head(5))
#del(data, train_sents, train_labels)

SENTENCE: Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .

SENTENCE LABEL: O O O O O O B-geo O O O O O B-geo O O O O O B-gpe O O O O O

ORIGINAL DATA:
     Sentence #           Word  POS Tag
0  Sentence: 1      Thousands  NNS   O
1          NaN             of   IN   O
2          NaN  demonstrators  NNS   O
3          NaN           have  VBP   O
4          NaN        marched  VBN   O


In [4]:
data = pd.read_csv("ner_dataset.csv", encoding = "ISO-8859-1").fillna(method='ffill')
data.tail(10)

,Sentence #,Word,POS,Tag
1048565,Sentence: 47958,impact,NN,O
1048566,Sentence: 47958,.,.,O
1048567,Sentence: 47959,Indian,JJ,B-gpe
1048568,Sentence: 47959,forces,NNS,O
1048569,Sentence: 47959,said,VBD,O
1048570,Sentence: 47959,they,PRP,O
1048571,Sentence: 47959,responded,VBD,O
1048572,Sentence: 47959,to,TO,O
1048573,Sentence: 47959,the,DT,O
1048574,Sentence: 47959,attack,NN,O


<a name="1.1"></a>
## 1.1  Importing the Data

In this part, we will import the preprocessed data and explore it.

```
def get_vocab(vocab_path, tags_path):
    vocab = {}
    with open(vocab_path) as f:
        for i, l in enumerate(f.read().splitlines()):
            vocab[l] = i  # to avoid the 0
        # loading tags (we require this to map tags to their indices)
    vocab['<PAD>'] = len(vocab) # 35180
    tag_map = {}
    with open(tags_path) as f:
        for i, t in enumerate(f.read().splitlines()):
            tag_map[t] = i 
    
    return vocab, tag_map
```

---

```
def get_params(vocab, tag_map, sentences_file, labels_file):
    sentences = []
    labels = []

    with open(sentences_file) as f:
        for sentence in f.read().splitlines():
            # replace each token by its index if it is in vocab
            # else use index of UNK_WORD
            s = [vocab[token] if token in vocab 
                 else vocab['UNK']
                 for token in sentence.split(' ')]
            sentences.append(s)

    with open(labels_file) as f:
        for sentence in f.read().splitlines():
            # replace each label by its index
            l = [tag_map[label] for label in sentence.split(' ')] # I added plus 1 here
            labels.append(l) 
    return sentences, labels, len(sentences)
```

**Encoding the words**  
The embedding lookup requires that we pass in integers to our network. The easiest way to do this is to create dictionaries that map the words in the vocabulary to integers. Then we can convert each of our reviews into integers so they can be passed into the network.

>Build a dictionary that maps words to integers. Later we're going to pad our input vectors with zeros, so make sure the integers **start at 1, not 0**.

In [5]:
from collections import Counter

In [6]:
## Build a dictionary that maps words to integers
words_count = Counter(data["Word"])
vocab_to_int = {w: (i+1) for i, w in enumerate(words_count)}
vocab_to_int['UNK'] = len(vocab_to_int)
#vocab_to_int['<PAD>'] = len(vocab_to_int)

In [7]:
dict(list(words_count.items())[:10])

{'Thousands': 114,
 'of': 26354,
 'demonstrators': 110,
 'have': 5485,
 'marched': 65,
 'through': 515,
 'London': 261,
 'to': 23213,
 'protest': 237,
 'the': 52573}

In [8]:
dict(list(vocab_to_int.items())[:10]), dict(list(vocab_to_int.items())[-5:])

({'Thousands': 1,
  'of': 2,
  'demonstrators': 3,
  'have': 4,
  'marched': 5,
  'through': 6,
  'London': 7,
  'to': 8,
  'protest': 9,
  'the': 10},
 {'hardliner': 35175,
  'indicative': 35176,
  '3700': 35177,
  'Bermel': 35178,
  'UNK': 35178})

In [9]:
# stats about vocabulary
print('Unique words: ', len(vocab_to_int))  # should ~ 74000+
print()

Unique words:  35179



In [10]:
## Build a dictionary that maps tags to integers
tag_to_int = {t: (i+1) for i, t in enumerate(set(data["Tag"]))}
tag_to_int

{'B-org': 1,
 'B-eve': 2,
 'I-tim': 3,
 'B-art': 4,
 'I-gpe': 5,
 'B-gpe': 6,
 'I-art': 7,
 'I-eve': 8,
 'I-nat': 9,
 'B-per': 10,
 'I-geo': 11,
 'I-per': 12,
 'I-org': 13,
 'O': 14,
 'B-geo': 15,
 'B-nat': 16,
 'B-tim': 17}

Convert the sentences to integers and store the sentences in a new list called `sents_ints`.

In [11]:
def get_params2(vocab_to_int, tag_to_int, df):
    sents_int = []
    labels_int = [] 
    
    sents_w = df.groupby('Sentence #')['Word'].apply(list).values
    for sent in sents_w:
        sent_int = [vocab_to_int[w] if w in vocab_to_int else vocab_to_int['UNK'] for w in sent]
        sents_int.append(sent_int)
    
    sents_tags = df.groupby('Sentence #')['Tag'].apply(list).values
    for sent_tags in sents_tags:
        label_int = [tag_to_int[t] for t in sent_tags]
        labels_int.append(label_int)
    
    return sents_int, labels_int, len(sents_int)

In [12]:
sents_int, labels_int, sent_size = get_params2(vocab_to_int, tag_to_int, data)

train-val-test split 7:1.5:1.5

In [13]:
split_frac1, split_frac2 = 0.7, 0.15

## split data into training, validation, and test data (sentences, labels, and len)
split_idx = int(len(sents_int)*split_frac1)
train_sent, train_label = sents_int[:split_idx], labels_int[:split_idx]

split_idx2 = split_idx + int(len(sents_int)*split_frac2)
valid_sent, valid_label = sents_int[split_idx:split_idx2], labels_int[split_idx:split_idx2]
test_sent, test_label = sents_int[split_idx2:], labels_int[split_idx2:]

train_len, valid_len, test_len = len(train_sent), len(valid_sent), len(test_sent)

In [14]:
train_len, valid_len, test_len

(33571, 7193, 7195)

In [15]:
train_sent[0]

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 10,
 16,
 2,
 17,
 18,
 19,
 20,
 21,
 22]

--

In [16]:
vocab, tag_map = get_vocab('data/large/words.txt', 'data/large/tags.txt')
t_sentences, t_labels, t_size = get_params(vocab, tag_map, 'data/large/train/sentences.txt', 'data/large/train/labels.txt')
v_sentences, v_labels, v_size = get_params(vocab, tag_map, 'data/large/val/sentences.txt', 'data/large/val/labels.txt')
test_sentences, test_labels, test_size = get_params(vocab, tag_map, 'data/large/test/sentences.txt', 'data/large/test/labels.txt')

In [17]:
t_size, v_size, test_size

(33570, 7194, 7194)

In [18]:
t_size + v_size + test_size

47958

In [19]:
tag_map

{'O': 0,
 'B-geo': 1,
 'B-gpe': 2,
 'B-per': 3,
 'I-geo': 4,
 'B-org': 5,
 'I-org': 6,
 'B-tim': 7,
 'B-art': 8,
 'I-art': 9,
 'I-per': 10,
 'I-gpe': 11,
 'I-tim': 12,
 'B-nat': 13,
 'B-eve': 14,
 'I-eve': 15,
 'I-nat': 16}

`vocab` is a dictionary that translates a word string to a unique number. Given a sentence, you can represent it as an array of numbers translating with this dictionary. The dictionary contains a `<PAD>` token. 

When training an LSTM using batches, all your input sentences must be the same size. To accomplish this, you set the length of your sentences to a certain number and add the generic `<PAD>` token to fill all the empty spaces. 

In [20]:
# vocab translates from a word to a unique number
print('vocab["the"]:', vocab["the"])
# Pad token
print('padded token:', vocab['<PAD>'])

vocab["the"]: 9
padded token: 35180


In [21]:
###
# vocab translates from a word to a unique number
print('vocab["the"]:', vocab_to_int["the"])

vocab["the"]: 10


The tag_map corresponds to one of the possible tags a word can have. Run the cell below to see the possible classes you will be predicting. The prepositions in the tags mean:
* I: Token is inside an entity.
* B: Token begins an entity.

In [22]:
print(tag_map)

{'O': 0, 'B-geo': 1, 'B-gpe': 2, 'B-per': 3, 'I-geo': 4, 'B-org': 5, 'I-org': 6, 'B-tim': 7, 'B-art': 8, 'I-art': 9, 'I-per': 10, 'I-gpe': 11, 'I-tim': 12, 'B-nat': 13, 'B-eve': 14, 'I-eve': 15, 'I-nat': 16}


In [23]:
###
print(tag_to_int)

{'B-org': 1, 'B-eve': 2, 'I-tim': 3, 'B-art': 4, 'I-gpe': 5, 'B-gpe': 6, 'I-art': 7, 'I-eve': 8, 'I-nat': 9, 'B-per': 10, 'I-geo': 11, 'I-per': 12, 'I-org': 13, 'O': 14, 'B-geo': 15, 'B-nat': 16, 'B-tim': 17}


So the coding scheme that tags the entities is a minimal one where B- indicates the first token in a multi-token entity, and I- indicates one in the middle of a multi-token entity. If you had the sentence 

**"Sharon flew to Miami on Friday"**

the outputs would look like:

```
Sharon B-per
flew   O
to     O
Miami  B-geo
on     O
Friday B-tim
```

your tags would reflect three tokens beginning with B-, since there are no multi-token entities in the sequence. But if you added Sharon's last name to the sentence: 

**"Sharon Floyd flew to Miami on Friday"**

```
Sharon B-per
Floyd  I-per
flew   O
to     O
Miami  B-geo
on     O
Friday B-tim
```

then your tags would change to show first "Sharon" as B-per, and "Floyd" as I-per, where I- indicates an inner token in a multi-token sequence.

In [24]:
# Exploring information about the data
print('The number of outputs is tag_map', len(tag_map))
# The number of vocabulary tokens (including <PAD>)
g_vocab_size = len(vocab)
print(f"Num of vocabulary words: {g_vocab_size}")
print('The vocab size is', len(vocab))
print('The training size is', t_size)
print('The validation size is', v_size)
print('An example of the first sentence is', t_sentences[0])
print('An example of its corresponding label is', t_labels[0])

The number of outputs is tag_map 17
Num of vocabulary words: 35181
The vocab size is 35181
The training size is 33570
The validation size is 7194
An example of the first sentence is [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 9, 15, 1, 16, 17, 18, 19, 20, 21]
An example of its corresponding label is [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0]


In [25]:
###
# Exploring information about the data
print('The number of outputs is tag_map', len(tag_to_int))
# The number of vocabulary tokens (including <PAD>)
g_vocab_size = len(vocab_to_int)
#print(f"Num of vocabulary words: {g_vocab_size}")
print(f'The vocab size is {g_vocab_size}')
print(f'The training size is {train_len}')
print(f'The validation size is {valid_len}')
print(f'The validation size is {test_len}')
print(f'An example of the first sentence is {train_sent[0]}')
print(f'An example of its corresponding label is {train_label[0]}')

The number of outputs is tag_map 17
The vocab size is 35179
The training size is 33571
The validation size is 7193
The validation size is 7195
An example of the first sentence is [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 10, 16, 2, 17, 18, 19, 20, 21, 22]
An example of its corresponding label is [14, 14, 14, 14, 14, 14, 15, 14, 14, 14, 14, 14, 15, 14, 14, 14, 14, 14, 6, 14, 14, 14, 14, 14]


So you can see that we have already encoded each sentence into a tensor by converting it into a number. We also have 16 possible classes, as shown in the tag map.


<a name="1.2"></a>
## 1.2  Data generator

In python, a generator is a function that behaves like an iterator. It will return the next item. Here is a [link](https://wiki.python.org/moin/Generators) to review python generators. 

In many AI applications it is very useful to have a data generator. You will now implement a data generator for our NER application.

<a name="ex01"></a>
### Exercise 01

**Instructions:** Implement a data generator function that takes in `batch_size, x, y, pad, shuffle` where x is a large list of sentences, and y is a list of the tags associated with those sentences and pad is a pad value. Return a subset of those inputs in a tuple of two arrays `(X,Y)`. Each is an array of dimension (`batch_size, max_len`), where `max_len` is the length of the longest sentence *in that batch*. You will pad the X and Y examples with the pad argument. If `shuffle=True`, the data will be traversed in a random form.

**Details:**

This code as an outer loop  
```
while True:  
...  
yield((X,Y))  
```

Which runs continuously in the fashion of generators, pausing when yielding the next values. We will generate a batch_size output on each pass of this loop.    

It has two inner loops. 
1. The first stores in temporal lists the data samples to be included in the next batch, and finds the maximum length of the sentences contained in it. By adjusting the length to include only the size of the longest sentence in each batch, overall computation is reduced. 

2. The second loop moves those inputs from the temporal list into NumPy arrays pre-filled with pad values.

There are three slightly out of the ordinary features. 
1. The first is the use of the NumPy `full` function to fill the NumPy arrays with a pad value. See [full function documentation](https://numpy.org/doc/1.18/reference/generated/numpy.full.html).

2. The second is tracking the current location in the incoming lists of sentences. Generators variables hold their values between invocations, so we create an `index` variable, initialize to zero, and increment by one for each sample included in a batch. However, we do not use the `index` to access the positions of the list of sentences directly. Instead, we use it to select one index from a list of indexes. In this way, we can change the order in which we traverse our original list, keeping untouched our original list.  

3. The third also relates to wrapping. Because `batch_size` and the length of the input lists are not aligned, gathering a batch_size group of inputs may involve wrapping back to the beginning of the input loop. In our approach, it is just enough to reset the `index` to 0. We can re-shuffle the list of indexes to produce different batches each time.

Batching

In [26]:
###
class EntityDataset:
    def __init__(self, sentences, labels, pad):
        # sentences: [[61, 249, 19, ..., 722, 21], [...]]
        # labels: [[15, 15, 15, ..., 15, 15], [...]]
        # pad - an integer representing a pad character
        self.sentences = sentences
        self.labels = labels
        self.max_len = max([len(i) for i in self.sentences])
        self.pad = pad
        
    def __len__(self):
        return len(self.sentences)
    
    def __getitem__(self, item):
        sentence = self.sentences[item] # get only one sentence
        labels = self.labels[item]
        
        ids = []
        target_tag = []
        
        mask = [1] * len(sentence)
        token_type_ids = [0] * len(sentence)
        
        # right pads to the max_len in dataset(or batch) at
        # the size of the longest sentence in each batch
        padding_len = self.max_len - len(sentence)
    
        sentence = sentence + ([self.pad] * padding_len)
        mask = mask + ([self.pad] * padding_len)
        # Define sentence A and B indices associated to 1st and 2nd sentences
        # [0, 0, ..., 1,...,1]
        token_type_ids = token_type_ids + ([self.pad] * padding_len)
        labels = labels + ([self.pad] * padding_len)
        
        return {
            "input_ids": torch.tensor(sentence, dtype=torch.long),
            "token_type_ids": torch.tensor(token_type_ids, dtype=torch.long),
            "attention_mask": torch.tensor(mask, dtype=torch.long),  
            "labels": torch.tensor(labels, dtype=torch.long)
        }

In [27]:
# Create Tensor datasets
# similar as TensorDataset
train_dataset = EntityDataset(
    sentences=train_sent,
    labels=train_label,
    pad=0
)

valid_dataset = EntityDataset(
    sentences=valid_sent,
    labels=valid_label,
    pad=0
)
test_dataset = EntityDataset(
    sentences=test_sent,
    labels=test_label,
    pad=0
)

In [28]:
train_dataset[0]

{'input_ids': tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 10, 16,  2,
         17, 18, 19, 20, 21, 22,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [29]:
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 8
TEST_BATCH_SIZE = 8

In [30]:
# num_workers as a positive integer will turn on multi-process data loading 
# with the specified number of loader worker processes.
train_dataloader = DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE,
                              shuffle=True, num_workers=4)
valid_dataloader = DataLoader(valid_dataset, batch_size=VALID_BATCH_SIZE, 
                              shuffle=True, num_workers=1)
test_dataloader = DataLoader(test_dataset, batch_size=TEST_BATCH_SIZE,  
                             shuffle=True, num_workers=1)

Take a look at a batch of data:

In [31]:
one_batch = next(iter(train_dataloader))
one_batch

{'input_ids': tensor([[  551,  1973,  4827,  ...,     0,     0,     0],
         [17194,   353,   969,  ...,     0,     0,     0],
         [   50,   127,    10,  ...,     0,     0,     0],
         ...,
         [   43,   127,    10,  ...,     0,     0,     0],
         [  346,  2873,     8,  ...,     0,     0,     0],
         [ 2642,  1565,    94,  ...,     0,     0,     0]]),
 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'labels': tensor([[14, 14, 14,  ...,  0,  0,  0],
         [14, 14, 14,  ...,  0,  0,  0],
         [14, 14, 14,  ...,  0,  0,  

In [32]:
print(one_batch['input_ids'], '\n')
print(f"Batch size: {len(one_batch['input_ids'])}")

tensor([[  551,  1973,  4827,  ...,     0,     0,     0],
        [17194,   353,   969,  ...,     0,     0,     0],
        [   50,   127,    10,  ...,     0,     0,     0],
        ...,
        [   43,   127,    10,  ...,     0,     0,     0],
        [  346,  2873,     8,  ...,     0,     0,     0],
        [ 2642,  1565,    94,  ...,     0,     0,     0]]) 

Batch size: 32


In [33]:
# UNQ_C1 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
# GRADED FUNCTION: data_generator
def data_generator(batch_size, x, y, pad, shuffle=False, verbose=False):
    '''
      Input: 
        batch_size - integer describing the batch size
        x - list containing sentences where words are represented as integers
        y - list containing tags associated with the sentences
        shuffle - Shuffle the data order
        pad - an integer representing a pad character
        verbose - Print information during runtime
      Output:
        a tuple containing 2 elements:
        X - np.ndarray of dim (batch_size, max_len) of padded sentences
        Y - np.ndarray of dim (batch_size, max_len) of tags associated with the sentences in X
    '''
    
    # count the number of lines in data_lines
    num_lines = len(x)
    
    # create an array with the indexes of data_lines that can be shuffled
    lines_index = [*range(num_lines)]
    
    # shuffle the indexes if shuffle is set to True
    if shuffle:
        rnd.shuffle(lines_index)
    
    index = 0 # tracks current location in x, y
    while True:
        buffer_x = [0] * batch_size # Temporal array to store the raw x data for this batch
        buffer_y = [0] * batch_size # Temporal array to store the raw y data for this batch
                
  ### START CODE HERE (Replace instances of 'None' with your code) ###
        
        # Copy into the temporal buffers the sentences in x[index : index + batch_size] 
        # along with their corresponding labels y[index : index + batch_size]
        # Find maximum length of sentences in x[index : index + batch_size] for this batch. 
        # Reset the index if we reach the end of the data set, and shuffle the indexes if needed.
        max_len = 0
        for i in range(batch_size):
             # if the index is greater than or equal to the number of lines in x
            if index >= num_lines:
                # then reset the index to 0
                index = 0
                # re-shuffle the indexes if shuffle is set to True
                if shuffle:
                    rnd.shuffle(lines_index)
            
            # The current position is obtained using `lines_index[index]`
            # Store the x value at the current position into the buffer_x
            buffer_x[i] = x[lines_index[index]]
            
            # Store the y value at the current position into the buffer_y
            buffer_y[i] = y[lines_index[index]]
            
            lenx = len(buffer_x[i])    #length of current x[]
            if lenx > max_len:
                max_len = lenx         #max_len tracks longest x[]
            
            # increment index by one
            index += 1


        # create X,Y, NumPy arrays of size (batch_size, max_len) 'full' of pad value
        X = np.full((batch_size, max_len), pad)
        Y = np.full((batch_size, max_len), pad)

        # copy values from lists to NumPy arrays. Use the buffered values
        for i in range(batch_size):
            # get the example (sentence as a tensor)
            # in `buffer_x` at the `i` index
            x_i = buffer_x[i]
            
            # similarly, get the example's labels
            # in `buffer_y` at the `i` index
            y_i = buffer_y[i]
            
            # Walk through each word in x_i
            for j in range(len(x_i)):
                # store the word in x_i at position j into X
                X[i, j] = x_i[j]
                
                # store the label in y_i at position j into Y
                Y[i, j] = y_i[j]

    ### END CODE HERE ###
        if verbose: print("index=", index)
        yield((X,Y))

In [34]:
batch_size = 5
mini_sentences = t_sentences[0: 8]
mini_labels = t_labels[0: 8]
dg = data_generator(batch_size, mini_sentences, mini_labels, vocab["<PAD>"], shuffle=False, verbose=True)
X1, Y1 = next(dg)
X2, Y2 = next(dg)
print(Y1.shape, X1.shape, Y2.shape, X2.shape)
print(X1[0][:], "\n", Y1[0][:])

index= 5
index= 2
(5, 30) (5, 30) (5, 30) (5, 30)
[    0     1     2     3     4     5     6     7     8     9    10    11
    12    13    14     9    15     1    16    17    18    19    20    21
 35180 35180 35180 35180 35180 35180] 
 [    0     0     0     0     0     0     1     0     0     0     0     0
     1     0     0     0     0     0     2     0     0     0     0     0
 35180 35180 35180 35180 35180 35180]


**Expected output:**   
```
index= 5
index= 2
(5, 30) (5, 30) (5, 30) (5, 30)
[    0     1     2     3     4     5     6     7     8     9    10    11
    12    13    14     9    15     1    16    17    18    19    20    21
 35180 35180 35180 35180 35180 35180] 
 [    0     0     0     0     0     0     1     0     0     0     0     0
     1     0     0     0     0     0     2     0     0     0     0     0
 35180 35180 35180 35180 35180 35180]  
```

In [35]:
###
BATCH_SIZE = 5
mini_sentences = train_sent[0: 8]
mini_labels = train_label[0: 8]
mini_dataset = EntityDataset(mini_sentences, mini_labels, pad=0)
mini_dataloader = DataLoader(mini_dataset, batch_size=BATCH_SIZE, shuffle=True)
batch1 = next(iter(mini_dataloader))
batch2 = next(iter(mini_dataloader))
print(batch1['input_ids'].shape, batch1['labels'].shape, batch2['input_ids'].shape, batch2['labels'].shape)
#print(Y1.shape, X1.shape, Y2.shape, X2.shape
print(batch1['input_ids'][0], "\n", batch1['labels'][0])
#print(X1[0][:], "\n", Y1[0][:])

torch.Size([5, 35]) torch.Size([5, 35]) torch.Size([5, 35]) torch.Size([5, 35])
tensor([943, 944, 364, 945, 345, 946,  12,  10, 947, 249, 201,  94, 252, 318,
        948, 194, 181, 949,   8,   4, 333,   8, 950, 135, 377, 306, 312,  12,
        951, 544, 545,  22,   0,   0,   0]) 
 tensor([14, 14, 17, 14, 14, 14, 14, 14, 15, 14, 14, 14, 14, 14,  1, 14, 14, 14,
        14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 15, 11, 14,  0,  0,  0])


<a name="2"></a>
# Part 2:  Building the model

You will now implement the model. You will be using Google's TensorFlow. Your model will be able to distinguish the following:
<table>
    <tr>
        <td>
<img src = 'ner1.png' width="width" height="height" style="width:500px;height:150px;"/>
        </td>
    </tr>
</table>

The model architecture will be as follows: 

<img src = 'ner2.png' width="width" height="height" style="width:600px;height:250px;"/>

Concretely: 

* Use the input tensors you built in your data generator
* Feed it into an Embedding layer, to produce more semantic entries
* Feed it into an LSTM layer
* Run the output through a linear layer
* Run the result through a log softmax layer to get the predicted class for each word.

Good news! We won't make you implement the LSTM unit drawn above. However, we will ask you to build the model. 

<a name="ex02"></a>
### Exercise 02

**Instructions:** Implement the initialization step and the forward function of your Named Entity Recognition system.  
Please utilize help function e.g. `help(tl.Dense)` for more information on a layer
   
- [tl.Serial](https://github.com/google/trax/blob/1372b903bb66b0daccee19fd0b1fdf44f659330b/trax/layers/combinators.py#L26): Combinator that applies layers serially (by function composition).
    - You can pass in the layers as arguments to `Serial`, separated by commas. 
    - For example: `tl.Serial(tl.Embeddings(...), tl.Mean(...), tl.Dense(...), tl.LogSoftmax(...))` 


-  [tl.Embedding](https://github.com/google/trax/blob/1372b903bb66b0daccee19fd0b1fdf44f659330b/trax/layers/core.py#L113): Initializes the embedding. In this case it is the dimension of the model by the size of the vocabulary. 
    - `tl.Embedding(vocab_size, d_feature)`.
    - `vocab_size` is the number of unique words in the given vocabulary.
    - `d_feature` is the number of elements in the word embedding (some choices for a word embedding size range from 150 to 300, for example).
    

-  [tl.LSTM](https://github.com/google/trax/blob/1372b903bb66b0daccee19fd0b1fdf44f659330b/trax/layers/rnn.py#L87):`Trax` LSTM layer of size d_model. 
    - `LSTM(n_units)` Builds an LSTM layer of n_cells.



-  [tl.Dense](https://github.com/google/trax/blob/1372b903bb66b0daccee19fd0b1fdf44f659330b/trax/layers/core.py#L28):  A dense layer.
    - `tl.Dense(n_units)`: The parameter `n_units` is the number of units chosen for this dense layer.  


- [tl.LogSoftmax](https://github.com/google/trax/blob/1372b903bb66b0daccee19fd0b1fdf44f659330b/trax/layers/core.py#L242): Log of the output probabilities.
    - Here, you don't need to set any parameters for `LogSoftMax()`.
 

- `init_hidden` should initialize the hidden and cell state of an lstm layer to all zeros, and move those state to GPU, if available.
 

**Online documentation and some references** 

- [tl.Serial](https://trax-ml.readthedocs.io/en/latest/trax.layers.html#module-trax.layers.combinators)

- [tl.Embedding](https://trax-ml.readthedocs.io/en/latest/trax.layers.html#trax.layers.core.Embedding)

-  [tl.LSTM](https://trax-ml.readthedocs.io/en/latest/trax.layers.html#trax.layers.rnn.LSTM)

-  [tl.Dense](https://trax-ml.readthedocs.io/en/latest/trax.layers.html#trax.layers.core.Dense)

- [tl.LogSoftmax](https://trax-ml.readthedocs.io/en/latest/trax.layers.html#trax.layers.core.LogSoftmax)


- [nn.Embedding](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html)
    - Inputs: arbitrary shape. we use vocab_size mostly
    - Outputs: (input shape, embedding_dim)

- [nn.LSTM](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html#torch.nn.LSTM)
    - Inputs: 1.input (n_batch, input_size)  2.(h_0, c_0) ((n_batch, hidden_dim), (n_batch, hidden_dim))
    - Outputs: 1.output (n_batch, hidden_dim)  2.(h_n, c_n) ((n_batch, hidden_dim), (n_batch, hidden_dim))

- [nn.Linear](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html#torch.nn.Linear)

- [nn.functional.log_softmax](https://pytorch.org/docs/stable/generated/torch.nn.functional.log_softmax.html)

- [nn.LogSoftmax](https://pytorch.org/docs/stable/generated/torch.nn.LogSoftmax.html)


* [What does `next(self.parameters()).data` mean?](https://discuss.pytorch.org/t/what-does-next-self-parameters-data-mean/1458/2)
* [Shape of pytorch model.parameter is inconsistent with how it's defined in the model](https://stackoverflow.com/questions/53462493/shape-of-pytorch-model-parameter-is-inconsistent-with-how-its-defined-in-the-mo)

In [36]:
class NER(nn.Module):
    """
      Input: 
        vocab_size - integer containing the size of the vocabulary
        d_model - integer describing the embedding size (it's equal to hidden_dim here)
      Output:
        model - a torch serial model
    """    
    def __init__(self, vocab_size, embedding_dim, hidden_dim, tags=tag_to_int):
        super(NER, self).__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.tags = tags
    
        # embedding layer  
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        #self.lstm = nn.LSTM(vocab_size, d_model, batch_first=True)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, len(self.tags))
    
    
    def forward(self, x, hidden):
        embed_input = self.embedding(x) # embed.shape: 
        # the hidden should be initiate by below method
        # lstm_out.shape:[batch_size, d_model] 
        lstm_out, hidden = self.lstm(embed_input, hidden) 
        out = F.log_softmax(self.fc(lstm_out), axis=1)
        
        return out, hidden
        
    
    def init_hidden(self, batch_size, device):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data # weight.shape is [len(vocab_size), d_model] here
        
        # weight.new() creates a tensor that has the same data type, same device as the 
        # produced parameter.
        hidden = (weight.new(batch_size, self.hidden_dim).zero_().to(device),
                  weight.new(batch_size, self.hidden_dim).zero_().to(device))
        
        return hidden

In [37]:
# Instantiate the model w/ parameters
vocab_size = len(vocab_to_int) + 1 # +1 for the 0 padding
embedding_dim = 200 
hidden_dim = 64
tags = tag_to_int

model = NER(vocab_size, embedding_dim, hidden_dim, tag_to_int)

print(model)

NER(
  (embedding): Embedding(35180, 200)
  (lstm): LSTM(200, 64, batch_first=True)
  (fc): Linear(in_features=64, out_features=17, bias=True)
)


**variables (or parameters)**

 - **~Embedding.weight** – the learnable weights of the module of shape (num_embeddings, embedding_dim)
 - **~LSTM.weight_ih_l[k]** – the learnable input-hidden weights of the $k^{th}$ layer (W_ii|W_if|W_ig|W_io), of shape (4 *hidden_size, input_size) for k = 0. Otherwise, the shape is (4 *hidden_size, num_directions * hidden_size)
 - **~LSTM.weight_hh_l[k]** – the learnable hidden-hidden weights of the $k^{th}$ layer (W_hi|W_hf|W_hg|W_ho), of shape (4 *hidden_size, hidden_size). If proj_size > 0 was specified, the shape will be (4 *hidden_size, proj_size).
 - **~LSTM.bias_ih_l[k]** – the learnable input-hidden bias of the $k^{th}$ layer (b_ii|b_if|b_ig|b_io), of shape (4 *hidden_size)
 - **~LSTM.bias_hh_l[k]** – the learnable hidden-hidden bias of the $k^{th}$ layer (b_hi|b_hf|b_hg|b_ho), of shape (4 *hidden_size)
 - **~LSTM.weight_hr_l[k]** – the learnable projection weights of the $k^{th}$ layer of shape (proj_size, hidden_size). Only present when proj_size > 0 was specified.

torch.Size([256, 64])  
256 = 4 * 64, but why it'll multiply 64 with 4? Is't because there're 4 gates in lstm.

In [38]:
for name, param in model.named_parameters():
    print(name, '\t', param.size())

embedding.weight 	 torch.Size([35180, 200])
lstm.weight_ih_l0 	 torch.Size([256, 200])
lstm.weight_hh_l0 	 torch.Size([256, 64])
lstm.bias_ih_l0 	 torch.Size([256])
lstm.bias_hh_l0 	 torch.Size([256])
fc.weight 	 torch.Size([17, 64])
fc.bias 	 torch.Size([17])


In [39]:
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

embedding.weight 	 torch.Size([35180, 200])
lstm.weight_ih_l0 	 torch.Size([256, 200])
lstm.weight_hh_l0 	 torch.Size([256, 64])
lstm.bias_ih_l0 	 torch.Size([256])
lstm.bias_hh_l0 	 torch.Size([256])
fc.weight 	 torch.Size([17, 64])
fc.bias 	 torch.Size([17])


In [40]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, '\n', param.size(), '\n', param.data, '\n\n')

embedding.weight 
 torch.Size([35180, 200]) 
 tensor([[-0.2052,  0.6658,  1.2044,  ..., -0.2152, -0.5420,  0.2550],
        [ 0.6301, -0.0563, -1.5983,  ..., -0.4648, -0.4349,  1.9664],
        [ 1.4734, -1.4515,  1.4665,  ..., -0.7324,  0.8249, -0.2460],
        ...,
        [-0.6196, -0.0993, -1.0652,  ...,  0.6127,  1.8177, -0.3873],
        [ 0.8532,  1.7603,  1.7364,  ..., -0.6712, -0.8450, -0.1781],
        [ 0.5690,  0.3155,  0.1205,  ..., -0.4387, -0.5492,  1.2585]]) 


lstm.weight_ih_l0 
 torch.Size([256, 200]) 
 tensor([[-0.0520, -0.1225, -0.0171,  ..., -0.0472,  0.0785, -0.0587],
        [ 0.0616,  0.1213, -0.0496,  ..., -0.0644, -0.0577, -0.0474],
        [ 0.1047,  0.0930,  0.0394,  ...,  0.1028, -0.1142, -0.0591],
        ...,
        [ 0.1175,  0.0913,  0.0104,  ..., -0.0026, -0.0795,  0.0922],
        [-0.0764,  0.0722,  0.0676,  ...,  0.0465, -0.0416, -0.0185],
        [ 0.0661,  0.1204, -0.0964,  ...,  0.0973,  0.0522,  0.1221]]) 


lstm.weight_hh_l0 
 torch.Size([256

<a name="3"></a>
# Part 3:  Train the Model 

This section will train your model.

In [41]:
from trax.supervised import training

rnd.seed(33)

batch_size = 64

# Create training data, mask pad id=35180 for training.
train_generator = trax.supervised.inputs.add_loss_weights(
    data_generator(batch_size, t_sentences, t_labels, vocab['<PAD>'], True),
    id_to_mask=vocab['<PAD>'])

# Create validation data, mask pad id=35180 for training.
eval_generator = trax.supervised.inputs.add_loss_weights(
    data_generator(batch_size, v_sentences, v_labels, vocab['<PAD>'], True),
    id_to_mask=vocab['<PAD>'])

In [42]:
train_generator

<generator object add_loss_weights at 0x7f801c1e0de0>

In [43]:
# UNQ_C3 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
# GRADED FUNCTION: train_model
def train_model(NER, train_generator, eval_generator, train_steps=1, output_dir='model'):
    '''
    Input: 
        NER - the model you are building
        train_generator - The data generator for training examples
        eval_generator - The data generator for validation examples,
        train_steps - number of training steps
        output_dir - folder to save your model
    Output:
        training_loop - a trax supervised training Loop
    '''
    ### START CODE HERE (Replace instances of 'None' with your code) ###
    train_task = training.TrainTask(
      train_generator, # A train data generator
      loss_layer = tl.CrossEntropyLoss(), # A cross-entropy loss function
      optimizer = trax.optimizers.Adam(0.01),  # The adam optimizer
    )

    eval_task = training.EvalTask(
      labeled_data = eval_generator, # A labeled data generator
      metrics = [tl.CrossEntropyLoss(), tl.Accuracy()], # Evaluate with cross-entropy loss and accuracy
      n_eval_batches = 10 # Number of batches to use on each evaluation
    )

    training_loop = training.Loop(
        NER, # A model to train
        train_task, # A train task
        eval_task = eval_task, # The evaluation task
        output_dir = output_dir) # The output directory

    # Train with train_steps
    training_loop.run(n_steps = train_steps)
    ### END CODE HERE ###
    return training_loop

In [44]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

No GPU available, training on CPU; consider making n_epochs very small.


In [45]:
###
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
#torch.cuda.get_device_name(0)

cpu


Define the train function and evaluation(validation) function

In [ ]:
def train_fn(data_loader, model, optimizer, device, batch_size=batch_size):
    model.train()
    # initialize hidden state
    h = model.init_hidden(batch_size)
    train_loss = 0
    train_losses = [] # to keep the losses of each epoch
    for data in tqdm(data_loader, total=len(data_loader)):
        for k, v in data.items():
            data[k] = v.to(device)
        
        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = model.init_hidden(batch_size, device)
        model.zero_grad()

In [ ]:
# training params
epochs = 10

batch_size = 64

In [ ]:
# move model to GPU, if available
model.to(device)

for e in range(epochs):
    mdoel.train()